In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime
import glob
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

In [2]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

files = glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\????\*_proj.tif')

In [3]:
folders = glob.glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\20*')

In [4]:
folders

['D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2000',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2001',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2002',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2003',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2004',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2005',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2006',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2007',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2008',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2009',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2010',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2011',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2012',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2013',
 'D:\\MAIAC_Data\\Europe\\h00v01\\Projected\\2014']

In [ ]:
for folder in folders:
    files = glob.glob('{fname}\*_proj.tif'.format(fname=folder))
    print('Processing {fname}'.format(fname=folder))
    print(files)

In [5]:
for folder in folders:
    files = glob.glob(r'{fname}\*_proj.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_AOT = xr.concat(list_of_das, 'time')
    MAIAC_AOT.attrs.clear()
    reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))
    ds = reordered_MAIAC_AOT.to_dataset(name = 'data')
    ds.to_netcdf(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\{fol_name}.nc'.format(fol_name=))
    all_data = xr.open_mfdataset('*.nc', chunks={'time':10}, concat_dim='time')
    alldata = all_data['data']
    print('Processed {fname}'.format(fname=folder))

KeyboardInterrupt: 